In [ ]:
import pathlib
import pandas as pd

datadir = pathlib.Path.cwd().parent / "data"

pd_production_missing_category = pd.read_parquet(datadir / "production_missing_category.parquet")
pd_customer                    = pd.read_parquet(datadir / "customer.parquet")

# 11章 カテゴリ
## 11-1 カテゴリ型への変換
### Q: 性別をカテゴリに適したデータ型に変換


#### Awesome

In [ ]:
(
    pd_customer
    .assign(sex_cat=lambda df: df.sex.astype("category"))
    [["sex", "sex_cat"]]
)

## 11-2 カテゴリ値の欠損処理
### Q: $k$-NNによるカテゴリ値の補完
#### Awesome

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# （1） 欠損していないデータと欠損しているデータを分割
df_notnull = pd_production_missing_category.loc[lambda df: df.type.notnull()]
df_null = pd_production_missing_category.loc[lambda df: df.type.isnull()]

# （2） k-NNモデルのオブジェクトを作成
knn = KNeighborsClassifier(n_neighbors=3)
cols = ["length", "thickness"]

# （3） 欠損していないデータを用いてk-NNモデルを学習
knn.fit(df_notnull[cols], df_notnull["type"])

# （4） 欠損しているデータを用いて予測値を計算
pred = knn.predict(df_null[cols])

# （5） 欠損していないデータと欠損補完したデータを結合
pd.concat([df_notnull, df_null.assign(type=pred)])

#### Awesome

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# （1） 欠損していないデータと欠損しているデータを分割
df_not_null_index = pd_production_missing_category.index[
        pd_production_missing_category.type.notnull()
    ]
df_null_index = pd_production_missing_category.index.difference(df_not_null_index)

# （2） k-NNモデルのオブジェクトを作成
knn = KNeighborsClassifier(n_neighbors=3)
cols = ["length", "thickness"]

# （3） 欠損していないデータを用いてk-NNモデルを学習
knn.fit(
    pd_production_missing_category.loc[df_not_null_index, cols],
    pd_production_missing_category.loc[df_not_null_index, "type"]
)

# （4） 欠損しているデータを用いて予測値を計算
pd_production_missing_category.loc[df_null_index, "type"] = (
    knn.predict(pd_production_missing_category.loc[df_null_index, cols])
)

# （5） データが保管されたか確認
pd_production_missing_category

## 11-3 カテゴリ値の集約
### Q: 年齢カテゴリの集約


#### Awesome

In [ ]:
(
    pd_customer
    .assign(
        age_cat=lambda df:
            (df.age / 10)
            .astype(int).astype("category")
            .cat.as_ordered()
            .mask(lambda s: s >= 6, 6)
    )
)

## 11-4 カテゴリ値の組み合わせ
### Q: 性別と年齢のカテゴリの組み合わせ


#### Awesome

In [ ]:
(
    pd_customer
    # （1） ageのカテゴリを作成
    .assign(age_cat=lambda df: (df.age / 10).astype(int))
    # （2） sexとageを結合したカテゴリを作成
    .assign(sex_age_cat=lambda df:
        (df.sex.fillna("null") + "_" + df.age_cat.astype("str")).astype("category"))
)

## 11-5 カテゴリ値の数値化
### Q: 性別のone-hotエンコーディング


#### Awesome

In [ ]:
from category_encoders.one_hot import OneHotEncoder

# （1） エンコーディングを行うオブジェクトを生成
encoder = OneHotEncoder(cols="sex", use_cat_names=True)
# （2） エンコーディングを実施
encoder.fit_transform(pd_customer)